In [51]:
# =============================================================================
# --- 1. 导入必要的库 ---
# =============================================================================
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from cspsa import  run_cspsa_simulation
from simulation_utils import calculate_cspsa_violation


In [53]:
H = qt.basis(2, 0)
V = qt.basis(2, 1)
# =============================================================================
# --- 3. CSPSA 算法实现 ---
# =============================================================================
    a= 1.0 , s=1.25, b=0.25, r =1/6.0
  def alpha(k):
        return a / (k + 1.0) ** s

    def c(k):
        return b / (k + 1.0) ** r

def run_cspsa_source_noise(hparams, iterations, sigma):
    """
    为子图(a)运行CSPSA：源不稳定性。
    在每次迭代中，都从一个正态分布中采样theta来生成新的量子态。
    """

    params = np.random.uniform(-1, 1, 10) + 1j * np.random.uniform(-1, 1, 10)
    params /= np.linalg.norm(params)



    history = []
    mean_theta = np.pi / 4

    for k in range(1, iterations + 1):
        # 1. 在当前迭代中生成一个带噪声的量子态
        theta_k = np.random.normal(loc=mean_theta, scale=sigma)
        psi_k = np.cos(theta_k) * qt.tensor(H, H) + np.sin(theta_k) * qt.tensor(V, V)
        rho_k = qt.ket2dm(psi_k.unit())

        # 2. 计算当前违背值
        val_k = calculate_cspsa_violation(rho_k, params)
        history.append(val_k)

        # 3. CSPSA步骤
        delta = np.random.choice([1, -1, 1j, -1j], size=10)
        ck = c(k)

        params_plus = params + ck * delta

        # 使用单边扰动进行梯度估计
        val_plus = calculate_cspsa_violation(rho_k, params_plus)

        gradient_estimate = (val_plus - val_k) / ck * np.conj(delta)

        # 4. 更新参数（目标是最大化，所以是加号）
        params += alpha(k) * gradient_estimate
        params /= np.linalg.norm(params)

    return history


def run_cspsa_multiple_inits(hparams, iterations, num_runs):
    """
    为子图(b)运行CSPSA：测量未对准（用多个初始条件展示鲁棒性）。
    """

    # 理想的、固定的最大纠缠态
    psi_ideal = (qt.tensor(H, H) + qt.tensor(V, V)).unit()
    rho_ideal = qt.ket2dm(psi_ideal.unit())

    all_runs_history = []

    for i in range(num_runs):
        print(f"Running simulation for initial condition {i+1}/{num_runs}...")
        params = np.random.uniform(-1, 1, 10) + 1j * np.random.uniform(-1, 1, 10)
        params /= np.linalg.norm(params)


        history = []
        for k in range(1, iterations + 1):
            val_k = calculate_cspsa_violation(rho_ideal, params)
            history.append(val_k)

            delta = np.random.choice([1, -1, 1j, -1j], size=10)
            ck = c(k)

            params_plus = params + ck * delta
            val_plus = calculate_cspsa_violation(rho_ideal, params_plus)

            gradient_estimate = (val_plus - val_k) / ck * np.conj(delta)

            params += alpha(k) * gradient_estimate
            params /= np.linalg.norm(params)

        all_runs_history.append(history)

    return all_runs_history


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [29]:

# =============================================================================
# --- 4. 主执行块 ---
# =============================================================================

# 根据文本描述设置超参数
# 注意: Mathematica代码中的第二部分使用了不同的超参数，我们采用文本中的
HPARAMS_A = HPARAMS_A = {'a': 1.0, 's': 1.25, 'b': 0.25, 'r': 1/6.0}
ITERATIONS_A = 150
SIGMA = np.pi / 50 # 对应 Mathematica 中的 NormalDistribution[pi/4, pi/50]

HPARAMS_A = {'a': 1.0, 's': 1.25, 'b': 0.25, 'r': 1/6.0}
ITERATIONS_B = 80
NUM_RUNS_B = 5

# --- 运行模拟 ---
print("--- Running simulation for Figure 4(a): Instability of source ---")
history_a = run_cspsa_source_noise(hparams=HPARAMS_A, iterations=ITERATIONS_A, sigma=SIGMA)
print("Done.")

print("\n--- Running simulation for Figure 4(b): Measurement misalignment ---")
histories_b = run_cspsa_multiple_inits(hparams=HPARAMS_B, iterations=ITERATIONS_B, num_runs=NUM_RUNS_B)
print("Done.")


# =============================================================================
# --- 5. 绘图 ---
# =============================================================================
print("\nGenerating Figure 4...")
plt.style.use('seaborn-v0_8-whitegrid')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), dpi=120)
fig.suptitle("CSPSA Robustness Analysis", fontsize=16)

# --- 子图 (a): 不稳定的源 ---
ax1.plot(history_a, label='Violation Value per Iteration', color='royalblue')
# 计算并绘制后半部分迭代的均值，以显示波动中心
mean_val = np.mean(history_a[len(history_a)//2:])
ax1.axhline(mean_val, color='firebrick', linestyle='--', label=f'Average Violation ≈ {mean_val:.3f}')
ax1.set_title("(a) Instability of Source", fontsize=14)
ax1.set_xlabel("Iteration", fontsize=12)
ax1.set_ylabel("Instrumental Violation", fontsize=12)
ax1.legend()
ax1.grid(True, which='both', linestyle='--', alpha=0.7)
# 根据文本描述，收敛发生在30-50次迭代，我们显示更多以观察
ax1.set_xlim(0, ITERATIONS_A)

# --- 子图 (b): 测量未对准 ---
for i, history in enumerate(histories_b):
    ax2.plot(history, label=f'Initial Condition {i+1}', alpha=0.8)
ax2.set_title("(b) Measurement Misalignment (Robustness to Inits)", fontsize=14)
ax2.set_xlabel("Iteration", fontsize=12)
ax2.set_ylabel("Instrumental Violation", fontsize=12)
ax2.legend()
ax2.grid(True, which='both', linestyle='--', alpha=0.7)
ax2.set_xlim(0, ITERATIONS_B)


plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("figure4_robustness_analysis.pdf")
plt.show()

--- Running simulation for Figure 4(a): Instability of source ---


TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'str'

In [ ]:
HPARAMS_A = {'a': 1.0, 's': 1.25, 'b': 0.25, 'r': 1/6.0}